In [2]:
import geopandas as gpd
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
pd.set_option('display.max_rows', 200)

In [53]:
#降水量、気温のカラムを追加する
def tempreture_rain(gdf, gdf_weather):
  #データフレームをジオデータフレームにする
  gdf_weather = gpd.GeoDataFrame(gdf_weather, geometry='geometry')

  #座標系を変更
  src_proj = 4612 # 変換前の座標系を指定
  dst_proj = 4326  # 変換後の座標系を指定
  # ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
  gdf_weather.crs = f'epsg:{src_proj}'  # 変換前座標を指定
  gdf_weather = gdf_weather.to_crs(epsg=dst_proj)  # 変換後座標に変換

  avg_rain = []
  max_rain = []
  max_temperature = []
  min_temperature = []
  for i, row in gdf_weather.iterrows():
      avg_rain.append(row.iloc[1:13].mean())
      max_rain.append(row.iloc[1:13].max())
      max_temperature.append(row.iloc[15:49:3].max())
      min_temperature.append(row.iloc[15:49:3].min())
  gdf_weather['avg_rainfall'] = avg_rain
  gdf_weather['max_rainfall'] = max_rain
  gdf_weather['max_temperature'] = max_temperature
  gdf_weather['min_temperature'] = min_temperature
  #100mメッシュに1kmメッシュを結合
  gdf_merge = gpd.sjoin(gdf, gdf_weather[['number', 'G02_053', 'max_temperature', 'min_temperature', 'avg_rainfall', 'max_rainfall', 'geometry']], how='left', op="intersects" )
  #結合の時に１kmメッシュの間にある100mのポリゴンは２回カウントされるのでindexが重複するものは削除する
  gdf_merge = gdf_merge[~gdf_merge.index.duplicated()]
  #結合するdataframeのindexが残っていると後で結合できないのでindex_rightを削除
  gdf_merge = gdf_merge.drop(columns=['index_right'])
  return gdf_merge

In [5]:
gdf_osyu_100m_mesh = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/gdf_osyu_100m_mesh.shp').filter(['Meshcode','geometry'])

In [6]:
#天気気温のデータをダウンロード
gdf_weather_5840 = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/平年値メッシュ/G02-22_5840-jgd_GML/G02-22_5840-jgd.shp')
gdf_weather_5841 = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/平年値メッシュ/G02-22_5841-jgd_GML/G02-22_5841-jgd.shp')

In [54]:
#天気気温の5840と5841を空間結合
gdf_weather = pd.concat([gdf_weather_5840, gdf_weather_5841], ignore_index=True)

In [55]:
number = [i for i in range(len(gdf_weather))]
gdf_weather['number'] = number

In [51]:
gdf_weather[gdf_weather['G02_001'] == '58403794']

,Meshcode,geometry_x,number,G02_053,max_temperature,min_temperature,avg_rainfall,max_rainfall,G02_001,geometry_y
318,5840379401,"POLYGON ((140.92625 38.99167, 140.92625 38.992...",3194,97,184,-50,1610.25,2525,58403794,"POLYGON ((140.92500 38.99167, 140.92500 39.000..."
319,5840379402,"POLYGON ((140.92750 38.99167, 140.92750 38.992...",3194,97,184,-50,1610.25,2525,58403794,"POLYGON ((140.92500 38.99167, 140.92500 39.000..."
320,5840379404,"POLYGON ((140.93000 38.99167, 140.93000 38.992...",3194,97,184,-50,1610.25,2525,58403794,"POLYGON ((140.92500 38.99167, 140.92500 39.000..."
321,5840379405,"POLYGON ((140.93125 38.99167, 140.93125 38.992...",3194,97,184,-50,1610.25,2525,58403794,"POLYGON ((140.92500 38.99167, 140.92500 39.000..."
322,5840379406,"POLYGON ((140.93250 38.99167, 140.93250 38.992...",3194,97,184,-50,1610.25,2525,58403794,"POLYGON ((140.92500 38.99167, 140.92500 39.000..."
323,5840379407,"POLYGON ((140.93375 38.99167, 140.93375 38.992...",3194,97,184,-50,1610.25,2525,58403794,"POLYGON ((140.92500 38.99167, 140.92500 39.000..."
324,5840379408,"POLYGON ((140.93500 38.99167, 140.93500 38.992...",3194,97,184,-50,1610.25,2525,58403794,"POLYGON ((140.92500 38.99167, 140.92500 39.000..."
325,5840379409,"POLYGON ((140.93625 38.99167, 140.93625 38.992...",3194,97,184,-50,1610.25,2525,58403794,"POLYGON ((140.92500 38.99167, 140.92500 39.000..."
326,5840379410,"POLYGON ((140.92500 38.99250, 140.92500 38.993...",3194,97,184,-50,1610.25,2525,58403794,"POLYGON ((140.92500 38.99167, 140.92500 39.000..."
327,5840379411,"POLYGON ((140.92625 38.99250, 140.92625 38.993...",3194,97,184,-50,1610.25,2525,58403794,"POLYGON ((140.92500 38.99167, 140.92500 39.000..."


In [56]:
#100mメッシュに気温のデータを結合
gdf_merge = tempreture_rain(gdf_osyu_100m_mesh, gdf_weather)

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [57]:
gdf_weather = pd.merge(gdf_merge, gdf_weather[['number', 'G02_001', 'geometry']], on='number')

In [ ]:
# 重複したインデックスを表示する
if gdf_weather.index.duplicated().any():
    print("重複したインデックス:", gdf_weather.index[gdf_weather.index.duplicated()])
else:
    print("重複したインデックスはありません。")

重複したインデックスはありません。


In [ ]:
gdf_weather.to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市　天気/奥州市_気候.csv')

In [48]:
gdf_weather[gdf_weather['number'] == 3184].to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市/奥州市　天気/奥州市_確認.csv')

In [58]:
gdf_weather[gdf_weather['G02_001'] == '58403794'].to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市/奥州市　天気/奥州市_天気_確認_v3.csv')